# VAR for Weekly

In [4]:
import pandas as pd
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.tools.eval_measures import rmse, aic

# Load your data
df = pd.read_csv('cleanmerged.csv')
df['Y'] = df['Y'].fillna(0)
# Make sure 'date' is datetime type
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)

# Test for stationarity using Augmented Dickey Fuller test
def adf_test(series, signif=0.05):
    result = adfuller(series, autolag='AIC')
    if result[1] <= signif:
        print(f"Stationary")
    else:
        print(f"Non-Stationary")
        
# Apply ADF test on each column
for name, column in df.iteritems():
    adf_test(column)
    
# If necessary, difference the data
df_transformed = df.diff().dropna()

# Apply ADF test again
for name, column in df_transformed.iteritems():
    adf_test(column)
    
# Choose order by inspecting partial autocorrelation plot
# Here, we are assuming order (p) as 1
model = VAR(df_transformed)
results = model.fit(maxlags=1)

# Print the summary
print(results.summary())

# Predict the next 10 steps ahead
lag_order = results.k_ar
forecast_input = df_transformed.values[-lag_order:]
forecast = pd.DataFrame(results.forecast(y=forecast_input, steps=10),
                        index=df.index[-10:],
                        columns=df.columns + '_1d')

print(forecast)


Non-Stationary
Stationary
Stationary
Stationary


/Users/leopan/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Mon, 29, May, 2023
Time:                     12:33:16
--------------------------------------------------------------------
No. of Equations:         2.00000    BIC:                   -12.5467
Nobs:                     322.000    HQIC:                  -12.5890
Log likelihood:           1123.55    FPE:                3.31494e-06
AIC:                     -12.6171    Det(Omega_mle):     3.25402e-06
--------------------------------------------------------------------
Results for equation Y
           coefficient       std. error           t-stat            prob
------------------------------------------------------------------------
const        -0.000116         0.001827           -0.064           0.949
L1.Y         -0.385106         0.051234           -7.517           0.000
L1.X          0.056065         0.028151            1.992           0.046

Results for equation X

# Granger Causality Future and ETF

In [7]:
import pandas as pd
from statsmodels.tsa.stattools import grangercausalitytests

# Assuming 'df' is your DataFrame
df = pd.read_csv('cleanmerged.csv')
df['Y'] = df['Y'].fillna(0)
df = df.sort_values('date')
df.set_index('date', inplace=True)

# Testing if 'X1' Granger-causes 'Y'
granger_test_result = grangercausalitytests(df[['Y', 'X']], maxlag=2)


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.0313  , p=0.8596  , df_denom=320, df_num=1
ssr based chi2 test:   chi2=0.0316  , p=0.8588  , df=1
likelihood ratio test: chi2=0.0316  , p=0.8588  , df=1
parameter F test:         F=0.0313  , p=0.8596  , df_denom=320, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.0880  , p=0.9158  , df_denom=317, df_num=2
ssr based chi2 test:   chi2=0.1788  , p=0.9145  , df=2
likelihood ratio test: chi2=0.1787  , p=0.9145  , df=2
parameter F test:         F=0.0880  , p=0.9158  , df_denom=317, df_num=2


# Granger Causality Future and Minning 

In [12]:
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import grangercausalitytests

# Load data from csv file into DataFrame
df = pd.read_csv("Future and Minning Clean.csv")

# Make sure your data is in date order before performing Granger causality test
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values('Date')
df.set_index('Date', inplace=True)

# Drop rows with NaN or inf values
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(inplace=True)

# List of stock variables
stocks = ['LAC', '002466.SZ', 'SQM', 'ALB']

# Perform Granger causality tests
for stock in stocks:
    print(f'\nGranger Causality of Y on {stock}:')
    for lag in [3, 4]:
        print(f'\nLag: {lag}')
        granger_test_result = grangercausalitytests(df[['Y', stock]], maxlag=lag)


Granger Causality of Y on LAC:

Lag: 3

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.1108  , p=0.7395  , df_denom=313, df_num=1
ssr based chi2 test:   chi2=0.1118  , p=0.7381  , df=1
likelihood ratio test: chi2=0.1118  , p=0.7381  , df=1
parameter F test:         F=0.1108  , p=0.7395  , df_denom=313, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.5004  , p=0.6068  , df_denom=310, df_num=2
ssr based chi2 test:   chi2=1.0170  , p=0.6014  , df=2
likelihood ratio test: chi2=1.0153  , p=0.6019  , df=2
parameter F test:         F=0.5004  , p=0.6068  , df_denom=310, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=0.3361  , p=0.7992  , df_denom=307, df_num=3
ssr based chi2 test:   chi2=1.0314  , p=0.7936  , df=3
likelihood ratio test: chi2=1.0297  , p=0.7941  , df=3
parameter F test:         F=0.3361  , p=0.7992  , df_denom=307, df_num=3

Lag: 4

Granger Causality
number of lags (no zero) 1
